# Truth Table Generator

This is a short introduction to truth tables.

# Table of Contents
1. [Basics](#Basics)<br>
    1.1 [Unary operator NOT](#Unary-operator-NOT)<br>
    2.1 [Binary operators](#Binary-operators)<br>
2. [Equivalences](#Equivalences)

In [1]:
from itertools import product
import pandas as pd

In [2]:
# A BoolFun takes a variable number of named arguments
# and returns a boolean value.
# Example: F = BoolFun('p^q', lambda **ps : ps['p'] and ps['q'])
#          is a function satisfying F(q=True, p=True) = True.
class BoolFun:
    def __init__(self, name, fun):
        self.name = name
        self.fun = fun

    def __call__(self, **ps):
        return self.fun(**ps)

    def pname(self):
        if len(self.name) == 1:
            return self.name
        else:
            return '(' + self.name + ')'

    def Not(self):
        return BoolFun('~' + self.pname(), lambda **ps : not self(**ps))

    def And(self, other):
        new_name = self.pname() + '^' + other.pname()
        return BoolFun(new_name, lambda **ps : self(**ps) and other(**ps))

    def Or(self, other):
        new_name = self.pname() + 'v' + other.pname()
        return BoolFun(new_name, lambda **ps : self(**ps) or other(**ps))

    def XOr(self, other):
        new_name = self.pname() + '+' + other.pname()
        return BoolFun(new_name, lambda **ps : self(**ps) != other(**ps))

    def Implies(self, other):
        new_name = self.pname() + '=>' + other.pname()
        return BoolFun(new_name, lambda **ps : (not self(**ps)) or other(**ps))

    def Iff(self, other):
        new_name = self.pname() + '<=>' + other.pname()
        return BoolFun(new_name, lambda **ps : self(**ps) == other(**ps))

# Candidates for variable names
def IsLowerAlpha(c):
    return len(c) == 1 and 'a' <= c and c <= 'z' and c != 'v'

# For nicer formatting of table headers
def TeXify(string):
    string = string.replace('<=>', '\\Leftrightarrow ')
    string = string.replace('=>', '\\Rightarrow ')
    string = string.replace('+', '\\oplus ')
    string = string.replace('v', '\\vee ')
    string = string.replace('^', '\\wedge ')
    string = string.replace('~', '\\neg ')
    return '$' + string + '$'

# Makes the zero-input BoolFuns `True` and `False`,
# and the single-input BoolFuns given by a single-letter variable.
# Example: P = PrimitiveFun('p') is a function satisfying P(p=True) = True.
#          The input argument must be named: P(True) will result in an error.
# Any malformed input yields the zero-input BoolFun `False`.
def PrimitiveFun(char):
    if char == 'T':
        return BoolFun('T', lambda **ps : True)
    elif IsLowerAlpha(char):
        return BoolFun(char, lambda **ps : ps[char])
    else:
        return BoolFun('F', lambda **ps : False)
        

In [3]:
# Creates a truth table from a variable number of BoolFuns.
def TruthTable(*boolfuns):
    varnames = sorted(list(set([c for bf in boolfuns for c in bf.name if IsLowerAlpha(c)])))
    A = []
    for ps in product([True, False], repeat=len(varnames)):
        kwargs = dict(zip(varnames, ps))
        A.append(list(ps) + [bf(**kwargs) for bf in boolfuns])
    column_names = varnames + [bf.name for bf in boolfuns]
    D = pd.DataFrame(A, columns=[TeXify(c) for c in column_names])
    D.replace({True: 'T', False: 'F'}, inplace=True)
    return D

## Basics

Here we'll just define a few basic variable names: P, Q, R, and the truth-values T and F.

In [4]:
P = PrimitiveFun('p')
Q = PrimitiveFun('q')
R = PrimitiveFun('r')
T = PrimitiveFun('T')
F = PrimitiveFun('F')

### Unary operator NOT

In [ ]:
TruthTable(P.Not())

### Binary operators

In [ ]:
TruthTable(P.And(Q), P.Or(Q), P.XOr(Q), P.Implies(Q), P.Iff(Q))

## Equivalences

In [ ]:
TruthTable(P.Not(), P.Not().Not())

In [ ]:
TruthTable(P.XOr(P))

### DeMorgan's Laws

In [ ]:
TruthTable(P.And(Q).Not(), P.Not().Or(Q.Not()), P.Or(Q).Not(), P.Not().And(Q.Not()))

### Implication

In [ ]:
TruthTable(P.Implies(Q), P.Not().Or(Q), P.And(Q.Not()).Not())

In [ ]:
TruthTable(P.Iff(Q), P.Implies(Q).And(Q.Implies(P)))

### Associativity

In [ ]:
TruthTable(P.And(Q.And(R)), P.And(Q).And(R), P.Or(Q.Or(R)), P.Or(Q).Or(R))

In [ ]:
TruthTable(P.Implies(Q.Implies(R)), P.Implies(Q).Implies(R))

## Reductions

In [ ]:
TruthTable(P.Not(), P.And(T), P.And(F), P.Or(T), P.Or(F), P.XOr(T), P.XOr(F))

In [ ]:
TruthTable(P.Not(), P.Implies(T), P.Implies(F), T.Implies(P), F.Implies(P))